In [3]:
import copy
import pickle
import time
import numpy as np
import seaborn as sns

from tensorflow.keras.datasets.fashion_mnist import load_data
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Input, Dense, Dropout, Flatten, Input, Activation, Conv2D, MaxPooling2D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping

# from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
def data_reshape(x):
    x = (x.reshape(len(x), 28 * 28)).astype(np.float64)
    x /= x.max()
    return x

def model_save(model, filename):
    pickle.dump(model, open(filename, 'wb'))

def model_load(filename):
    return pickle.load(open(filename, 'rb'))

def save_accuracy(accuracy, filename):
    with open(filename, mode='w') as f:
        for d in accuracy:
            f.write("%s\n" % d)
    
def save_times(times, filename):
    with open(filename, mode='w') as f:
        for d in times:
            f.write("%s\n" % d)

In [ ]:
# 何%を検証用にするか
nVals = np.array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0])

# train_test_split関数に与える乱数の種値
random_state = 60

class_num = len([x for x in range(10)])

(x_train, y_train), (x_test, y_test) = load_data()
x_train, _, y_train, _ = train_test_split(x_train, y_train, test_size=0.9, random_state=random_state)
_, x_test, _, y_test = train_test_split(x_test, y_test, test_size=0.1, random_state=random_state)

class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

# x_train = x_train.astype('float32')
# x_train = x_train / 255.0
# x_test = x_test.astype('float32')
# x_test = x_test / 255.0

x_train = data_reshape(x_train)
x_test = data_reshape(x_test)

# one-hot表現に変換
# y_train = to_categorical( y_train, class_num )
# y_test = to_categorical( y_test, class_num )

In [ ]:
accuracies = []  # 正確度保存用
proc_time = []   # 実行時間保存用

tree_numbers = [x for x in range(1, 11)]

for tree_number in tree_numbers:
    start = time.time()
    model = RandomForestClassifier(n_estimators=tree_number, random_state=random_state)
    model.fit(x_train, y_train)
    duration = time.time() - start
    score = model.score(x_test, y_test)
    
    accuracies.append(score)
    proc_time.append(duration)
    
    filename = './Results/RandomForest/Models/randomforest-datalength=' + str(len(x_train)) + '-tree_number=' + str(tree_number) +'.pkl'
    model_save(model, filename)
    
filename = './Results/RandomForest/accuracy/randomforest-datalength=' + str(len(x_train)) + '-accuracy.txt'
save_accuracy(accuracies, filename)

filename = './Results/RandomForest/time/randomforest-datalength=' + str(len(x_train)) + '-time.txt'
save_times(proc_time, filename)

In [ ]:
# earlystopping = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=5, verbose=1)
# model = use_vgg16( class_num )
# print("コンパイルします")
# model.compile(optimizer=Adam( learning_rate=1e-4 ), loss='categorical_crossentropy', metrics=['accuracy'])
# print("学習を開始します")
# history = model.fit( x=x_train, y=y_train, batch_size=4, epochs=30, verbose=1, validation_data=(x_test, y_test), callbacks=[earlystopping])
# print("モデルを保存します")
# model_save(history, './Results/vgg16-{}'.format(len(x_train)))

In [ ]:
x_train.shape[1]

In [ ]:
min(y_test)

In [ ]:
max(y_test)

In [ ]:
plt.figure(figsize=(14,10))

# 学習用データの中から乱数で適当に．．．
for i in range(5):
    for j in range(10):
        num = np.random.randint(0, high=len(y_train) )
        image = x_train[num]
        ax = plt.subplot(5, 10, 1 + 10 * i + j)
        ax.set_axis_off()
        ax.set_title( "{}".format(class_names[y_train[num]]))
        plt.imshow(image.reshape(28, 28), cmap=plt.cm.binary)
plt.show()

In [ ]:
def create_model( WIDTH, HEIGHT, class_num ):
    # モデルを定義
    input_tensor = Input( shape=(HEIGHT, WIDTH, 3) )
    vgg = VGG16( include_top=False, input_tensor=input_tensor, weights=None )
    x = vgg.output
    x = Flatten()(x)
    x = Dense( 2048, activation="relu" )(x)
    x = Dropout(0.5)(x)
    x = Dense( 2048, activation="relu" )(x)
    x = Dropout(0.5)(x)
    x = Dense( class_num )(x)
    x = Activation( "softmax" )(x)
    return Model( inputs=vgg.inputs, outputs=x )

model = create_model( 400, 600, 7 )
model.summary()

In [ ]:
def use_vgg16(class_num):
    input_tensor = Input( shape=(32, 32, 1) )
    vgg = VGG16( include_top=False, input_tensor=input_tensor, weights=None, classes=class_num )
    x = vgg.output
    x = Flatten()(x)
    x = Dense( class_num, activation="relu" )(x)
    x = Dropout(0.5)(x)
    x = Activation( "softmax" )(x)
    return Model( inputs=vgg.inputs, outputs=x )

model = use_vgg16(10)
model.summary()

In [ ]:
from tensorflow.keras.applications.resnet50 import ResNet50
def create_model( WIDTH, HEIGHT, class_num ):
    # モデルを定義
    input_tensor = Input( shape=(HEIGHT, WIDTH, 3) )
    res = ResNet50( include_top=True, input_tensor=input_tensor, weights=None, classes=7 )
    return Model( inputs=res.inputs, outputs=res.output )
model = create_model( 400, 600, 7 )
model.summary()

In [4]:
# train_test_split関数に与える乱数の種値
random_state = 60

class_num = len([x for x in range(10)])

(x_train, y_train), (x_test, y_test) = load_data()
x_train, _, y_train, _ = train_test_split(x_train, y_train, test_size=0.9, random_state=random_state)
_, x_test, _, y_test = train_test_split(x_test, y_test, test_size=0.1, random_state=random_state)

class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

x_train = x_train.astype('float32')
x_train = x_train / 255.0
x_test = x_test.astype('float32')
x_test = x_test / 255.0

# one-hot表現に変換
y_train = to_categorical( y_train, class_num )
y_test = to_categorical( y_test, class_num )

In [ ]:
len(x_train)

In [ ]:
def model_save( model, history, filename ):
    base_path = './Results/NN/'
    h5 = base_path + 'Models/' + filename + '.h5'
    weights = base_path + 'weights/' + filename + '-weights.h5'
    pkl = base_path + 'history/'+ filename +'.pkl'
    try:
        # 保存
        model.save( h5 )
        model.save_weights( weights )

        with open( pkl, 'wb' ) as h_file:
            pickle.dump(history.history, h_file)
        print("学習結果の保存が完了しました。")
    except:
        print("保存に失敗しました。")

def create_model(class_num):
    model = Input( shape=(28, 28, 1) )
    x = Dense(100, activation="relu")(model)
    x = Dense(100, activation="relu")(x)
    x = Dense(100, activation="relu")(x)
    x = Dense(100, activation="relu")(x)
    x = Dense(100, activation="relu")(x)
    x = Flatten()(x)
    output = Dense(class_num, activation="softmax")(x)
    return Model( inputs=model, outputs=output )

earlystopping = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=5, verbose=1)
model = create_model( class_num )
print("コンパイルします")
model.compile(optimizer=Adam( learning_rate=1e-4 ), loss='categorical_crossentropy', metrics=['accuracy'])
print("学習を開始します")
history = model.fit( x=x_train, y=y_train, batch_size=4,
                     epochs=50, verbose=1, validation_data=(x_test, y_test),
                   )
print("モデルを保存します")
model_save(model, history, 'NN=100×5-data={}-epochs=50'.format(len(x_train)))

In [5]:
def model_save( model, history, filename ):
    base_path = './Results/NN/'
    h5 = base_path + 'Models/' + filename + '.h5'
    weights = base_path + 'weights/' + filename + '-weights.h5'
    pkl = base_path + 'history/'+ filename +'.pkl'
    try:
        # 保存
        model.save( h5 )
        model.save_weights( weights )

        with open( pkl, 'wb' ) as h_file:
            pickle.dump(history.history, h_file)
        print("学習結果の保存が完了しました。")
    except:
        print("保存に失敗しました。")

def create_model(class_num):
    model = Input( shape=(28, 28, 1) )
    x = Dense(100, activation="relu")(model)
    x = Dense(100, activation="relu")(x)
    x = Dense(100, activation="relu")(x)
    x = Dense(100, activation="relu")(x)
    x = Dense(100, activation="relu")(x)
    x = Flatten()(x)
    output = Dense(class_num, activation="softmax")(x)
    return Model( inputs=model, outputs=output )

earlystopping = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=5, verbose=1)
model = create_model( class_num )
print("コンパイルします")
model.compile(optimizer=Adam( learning_rate=1e-4 ), loss='categorical_crossentropy', metrics=['accuracy'])
print("学習を開始します")
history = model.fit( x=x_train, y=y_train, batch_size=4,
                     epochs=7, verbose=1, validation_data=(x_test, y_test),
                   )
print("モデルを保存します")
model_save(model, history, 'NN=100×5-data={}-epochs=7'.format(len(x_train)))

コンパイルします
学習を開始します
Epoch 1/7
1500/1500 [==============================] - 19s 13ms/step - loss: 0.6844 - accuracy: 0.7553 - val_loss: 0.5181 - val_accuracy: 0.8130
Epoch 2/7
1500/1500 [==============================] - 21s 14ms/step - loss: 0.4746 - accuracy: 0.8325 - val_loss: 0.4679 - val_accuracy: 0.8370
Epoch 3/7
1500/1500 [==============================] - 21s 14ms/step - loss: 0.4227 - accuracy: 0.8490 - val_loss: 0.5373 - val_accuracy: 0.8150
Epoch 4/7
1500/1500 [==============================] - 21s 14ms/step - loss: 0.3894 - accuracy: 0.8585 - val_loss: 0.4612 - val_accuracy: 0.8260
Epoch 5/7
1500/1500 [==============================] - 23s 15ms/step - loss: 0.3674 - accuracy: 0.8637 - val_loss: 0.4161 - val_accuracy: 0.8540
Epoch 6/7
1500/1500 [==============================] - 23s 16ms/step - loss: 0.3426 - accuracy: 0.8760 - val_loss: 0.4615 - val_accuracy: 0.8270
Epoch 7/7
1500/1500 [==============================] - 22s 15ms/step - loss: 0.3268 - accuracy: 0.8807 - val_los

In [ ]:
def create_model(class_num):
    model = Input( shape=(28, 28, 1) )
    x = Conv2D(1, kernel_size=(3, 3), activation="relu", input_shape=(28, 28, 1))(model)
    x = MaxPooling2D(pool_size=(3, 3))(x)
    x = Flatten()(x)
    output = Dense(class_num, activation="softmax")(x)
    return Model( inputs=model, outputs=output )

model = create_model(10)
model.summary()

earlystopping = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=5, verbose=1)
model = create_model( class_num )
print("コンパイルします")
model.compile(optimizer=Adam( learning_rate=1e-4 ), loss='categorical_crossentropy', metrics=['accuracy'])
print("学習を開始します")
history = model.fit( x=x_train, y=y_train, batch_size=4,
                     epochs=50, verbose=1, validation_data=(x_test, y_test),
                   )
print("モデルを保存します")
model_save(model, history, 'NN=100×5-data={}-epochs=50-conv×1'.format(len(x_train)))

In [ ]:
def create_model(class_num):
    model = Input( shape=(28, 28, 1) )
    x = Conv2D(14, kernel_size=(3, 3), activation="relu", input_shape=(28, 28, 1))(model)
    x = MaxPooling2D(pool_size=(3, 3))(x)
    x = Conv2D(1, kernel_size=(3, 3), activation="relu")(x)
    x = MaxPooling2D(pool_size=(3, 3))(x)
    x = Flatten()(x)
    output = Dense(class_num, activation="softmax")(x)
    return Model( inputs=model, outputs=output )

model = create_model(10)
model.summary()

earlystopping = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=5, verbose=1)
model = create_model( class_num )
print("コンパイルします")
model.compile(optimizer=Adam( learning_rate=1e-4 ), loss='categorical_crossentropy', metrics=['accuracy'])
print("学習を開始します")
history = model.fit( x=x_train, y=y_train, batch_size=4,
                     epochs=50, verbose=1, validation_data=(x_test, y_test),
                   )
print("モデルを保存します")
model_save(model, history, 'NN=100×5-data={}-epochs=50-conv×2'.format(len(x_train)))

In [ ]:
def model_save( model, history, filename ):
    base_path = './Results/NN/'
    h5 = base_path + 'Models/' + filename + '.h5'
    weights = base_path + 'weights/' + filename + '-weights.h5'
    pkl = base_path + 'history/'+ filename +'.pkl'
    try:
        # 保存
        model.save( h5 )
        model.save_weights( weights )

        with open( pkl, 'wb' ) as h_file:
            pickle.dump(history.history, h_file)
        print("学習結果の保存が完了しました。")
    except:
        print("保存に失敗しました。")

def create_model(class_num):
    model = Input( shape=(28, 28, 1) )
    x = Dense(100, activation="relu")(model)
    x = Dense(100, activation="relu")(x)
    x = Dense(100, activation="relu")(x)
    x = Dense(100, activation="relu")(x)
    x = Dropout(0.5)(x)
    x = Dense(100, activation="relu")(x)
    x = Flatten()(x)
    output = Dense(class_num, activation="softmax")(x)
    return Model( inputs=model, outputs=output )

earlystopping = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=5, verbose=1)
model = create_model( class_num )
print("コンパイルします")
model.compile(optimizer=Adam( learning_rate=1e-4 ), loss='categorical_crossentropy', metrics=['accuracy'])
print("学習を開始します")
history = model.fit( x=x_train, y=y_train, batch_size=4,
                     epochs=50, verbose=1, validation_data=(x_test, y_test),
                   )
print("モデルを保存します")
model_save(model, history, 'NN=100×5-dropout×1-data={}-epochs=50'.format(len(x_train)))

In [ ]:
def create_model(class_num):
    model = Input( shape=(28, 28, 1) )
    x = Dense(100, activation="relu")(model)
    x = Dense(100, activation="relu")(x)
    x = Dense(100, activation="relu")(x)
    x = Dropout(0.5)(x)
    x = Dense(100, activation="relu")(x)
    x = Dropout(0.5)(x)
    x = Dense(100, activation="relu")(x)
    x = Flatten()(x)
    output = Dense(class_num, activation="softmax")(x)
    return Model( inputs=model, outputs=output )

earlystopping = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=5, verbose=1)
model = create_model( class_num )
print("コンパイルします")
model.compile(optimizer=Adam( learning_rate=1e-4 ), loss='categorical_crossentropy', metrics=['accuracy'])
print("学習を開始します")
history = model.fit( x=x_train, y=y_train, batch_size=4,
                     epochs=50, verbose=1, validation_data=(x_test, y_test),
                   )
print("モデルを保存します")
model_save(model, history, 'NN=100×5-dropout×2-data={}-epochs=50'.format(len(x_train)))

In [ ]:
def create_model(class_num):
    model = Input( shape=(28, 28, 1) )
    x = Conv2D(16, kernel_size=(3, 3), activation="relu", input_shape=(28, 28, 1))(model)
    x = MaxPooling2D(pool_size=(3, 3))(x)
    x = Conv2D(8, kernel_size=(3, 3), activation="relu")(x)
    x = MaxPooling2D(pool_size=(3, 3))(x)
    x = Flatten()(x)
    output = Dense(class_num, activation="softmax")(x)
    return Model( inputs=model, outputs=output )

model = create_model(10)
model.summary()

earlystopping = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=5, verbose=1)
model = create_model( class_num )
print("コンパイルします")
model.compile(optimizer=Adam( learning_rate=1e-4 ), loss='categorical_crossentropy', metrics=['accuracy'])
print("学習を開始します")
history = model.fit( x=x_train, y=y_train, batch_size=4,
                     epochs=50, verbose=1, validation_data=(x_test, y_test),
                   )
print("モデルを保存します")
model_save(model, history, 'NN=100×5-data={}-epochs=50-conv×2_model=2'.format(len(x_train)))

In [ ]:
def model_save( model, history, filename ):
    base_path = './Results/NN/'
    h5 = base_path + 'Models/' + filename + '.h5'
    weights = base_path + 'weights/' + filename + '-weights.h5'
    pkl = base_path + 'history/'+ filename +'.pkl'
    try:
        # 保存
        model.save( h5 )
        model.save_weights( weights )

        with open( pkl, 'wb' ) as h_file:
            pickle.dump(history.history, h_file)
        print("学習結果の保存が完了しました。")
    except:
        print("保存に失敗しました。")

def create_model(class_num):
    model = Input( shape=(28, 28, 1) )
    x = Conv2D(1, kernel_size=(3, 3), activation="relu", input_shape=(28, 28, 1))(model)
    x = MaxPooling2D(pool_size=(3, 3))(x)
    x = Flatten()(x)
    x = Dense(100, activation="relu")(x)
    x = Dropout(0.5)(x)
    x = Dense(100, activation="relu")(x)
    x = Dropout(0.5)(x)
    x = Dense(100, activation="relu")(x)
    x = Dropout(0.5)(x)
    x = Dense(100, activation="relu")(x)
    x = Dropout(0.5)(x)
    x = Dense(100, activation="relu")(x)
    x = Dropout(0.5)(x)
    output = Dense(class_num, activation="softmax")(x)
    return Model( inputs=model, outputs=output )

model = create_model(10)
model.summary()

earlystopping = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=10, verbose=1)
model = create_model( class_num )
print("コンパイルします")
model.compile(optimizer=Adam( learning_rate=1e-4 ), loss='categorical_crossentropy', metrics=['accuracy'])
print("学習を開始します")
history = model.fit( x=x_train, y=y_train, batch_size=4,
                     epochs=200, verbose=1, validation_data=(x_test, y_test),
                     callbacks=[earlystopping]
                   )
print("モデルを保存します")
model_save(model, history, 'NN=100×5-dropout×5-data={}-epochs=200-conv×1'.format(len(x_train)))

In [ ]:
def model_save( model, history, filename ):
    base_path = './Results/NN/'
    h5 = base_path + 'Models/' + filename + '.h5'
    weights = base_path + 'weights/' + filename + '-weights.h5'
    pkl = base_path + 'history/'+ filename +'.pkl'
    try:
        # 保存
        model.save( h5 )
        model.save_weights( weights )

        with open( pkl, 'wb' ) as h_file:
            pickle.dump(history.history, h_file)
        print("学習結果の保存が完了しました。")
    except:
        print("保存に失敗しました。")

def create_model(class_num):
    model = Input( shape=(28, 28, 1) )
    x = Conv2D(1, kernel_size=(3, 3), activation="relu", input_shape=(28, 28, 1))(model)
    x = MaxPooling2D(pool_size=(3, 3))(x)
    x = Flatten()(x)
    x = Dense(100, activation="relu")(x)
    x = Dropout(0.5)(x)
    x = Dense(100, activation="relu")(x)
    x = Dropout(0.5)(x)
    x = Dense(100, activation="relu")(x)
    x = Dropout(0.5)(x)
    x = Dense(100, activation="relu")(x)
    x = Dropout(0.5)(x)
    x = Dense(100, activation="relu")(x)
    x = Dropout(0.5)(x)
    output = Dense(class_num, activation="softmax")(x)
    return Model( inputs=model, outputs=output )

model = create_model(10)
model.summary()

earlystopping = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=10, verbose=1)
model = create_model( class_num )
print("コンパイルします")
model.compile(optimizer=Adam( learning_rate=1e-4 ), loss='categorical_crossentropy', metrics=['accuracy'])
print("学習を開始します")
history = model.fit( x=x_train, y=y_train, batch_size=4,
                     epochs=200, verbose=1, validation_data=(x_test, y_test),
                     callbacks=[earlystopping]
                   )
print("モデルを保存します")
model_save(model, history, 'NN=100×5-dropout×5-data={}-epochs=200-conv×1'.format(len(x_train)))

In [ ]:
def model_save( model, history, filename ):
    base_path = './Results/NN/'
    h5 = base_path + 'Models/' + filename + '.h5'
    weights = base_path + 'weights/' + filename + '-weights.h5'
    pkl = base_path + 'history/'+ filename +'.pkl'
    try:
        # 保存
        model.save( h5 )
        model.save_weights( weights )

        with open( pkl, 'wb' ) as h_file:
            pickle.dump(history.history, h_file)
        print("学習結果の保存が完了しました。")
    except:
        print("保存に失敗しました。")

def create_model(class_num):
    model = Input( shape=(28, 28, 1) )
    x = Conv2D(16, kernel_size=(3, 3), activation="relu", input_shape=(28, 28, 1))(model)
    x = Conv2D(32, kernel_size=(3, 3), activation="relu")(x)
    x = Conv2D(1, kernel_size=(3, 3), activation="relu")(x)
    x = MaxPooling2D(pool_size=(3, 3))(x)
    x = Flatten()(x)
    x = Dense(100, activation="relu")(x)
    x = Dropout(0.5)(x)
    x = Dense(100, activation="relu")(x)
    x = Dropout(0.5)(x)
    x = Dense(100, activation="relu")(x)
    x = Dropout(0.5)(x)
    x = Dense(100, activation="relu")(x)
    x = Dropout(0.5)(x)
    x = Dense(100, activation="relu")(x)
    x = Dropout(0.5)(x)
    output = Dense(class_num, activation="softmax")(x)
    return Model( inputs=model, outputs=output )

model = create_model(10)
model.summary()

earlystopping = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=10, verbose=1)
model = create_model( class_num )
print("コンパイルします")
model.compile(optimizer=Adam( learning_rate=1e-4 ), loss='categorical_crossentropy', metrics=['accuracy'])
print("学習を開始します")
history = model.fit( x=x_train, y=y_train, batch_size=4,
                     epochs=200, verbose=1, validation_data=(x_test, y_test),
                     callbacks=[earlystopping]
                   )
print("モデルを保存します")
model_save(model, history, 'NN=100×5-dropout×5-data={}-epochs=200-conv×3'.format(len(x_train)))